In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

global num
num=str(4)
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.loadtxt('distance_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['distance_matrix']=np.dot(data['distance_matrix'],np.diag([1000000000000000] * len(data['distance_matrix'])))
    demands=[]
    for i in np.loadtxt('demand_'+str(num)+'.txt', dtype=np.float32, delimiter=','):
        demands.append(int(i[1]))
    data['demands'] = demands
    
#     sss=np.loadtxt('solution_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
#     for i in range(len(sss)):
#         sss[i]=sss[i].move(1)
#         sss[i]=sss[i].move(1)
#     for i in range(len(sss)):
#         for j in range(len(sss[i])):
#             sss[i][j]=sss[i][j]-1
#     data['initial_routes'] = sss
    
    
    w=np.loadtxt('spec_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['vehicle_capacities'] = []
    data['num_vehicles'] = int(w[1])
    for j in range(int(w[1])):
        data['vehicle_capacities'].append(int(w[2]))
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    
    f=open("solution_"+num+".txt","w")
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        l=[]
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            l.append(int('{0}'.format(node_index, route_load)))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        for i in range(len(l)):
            l[i]=l[i]+1
        l.append(1)
        kk=0
        for i in l:
            kk+=1
            if kk==len(l):
                f.write(str(i))
            else:
                f.write(str(i)+',')
        f.write('\n')
        print(l)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    f.close()


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

#     initial_solution = routing.ReadAssignmentFromRoutes(data['initial_routes'],
#                                                         True)
#     print('Initial solution:')
#     print_solution(data, manager, routing, initial_solution)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC)
    
# LOCAL_CHEAPEST_INSERTION\、PATH_CHEAPEST_ARC、PATH_MOST_CONSTRAINED_ARC、EVALUATOR_STRATEGY
# SAVINGS SWEEP CHRISTOFIDES ALL_UNPERFORMED BEST_INSERTION PARALLEL_CHEAPEST_INSERTION
# GLOBAL_CHEAPEST_ARC LOCAL_CHEAPEST_ARC FIRST_UNBOUND_MIN_VALUE    

#     search_parameters.local_search_metaheuristic = (
#         routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
#     search_parameters.time_limit.FromSeconds(1)
    
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 6000
    search_parameters.log_search = True
    search_parameters.use_full_propagation=True
    
        # Set default search parameters.
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    # Solve the problem.
#     solution = routing.SolveFromAssignmentWithParameters(
#         initial_solution, search_parameters)


#     # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 241973533391952506
[1, 37, 15, 20, 12, 19, 36, 1]
Route for vehicle 0:
 0 Load(0) ->  36 Load(450) ->  14 Load(1001) ->  19 Load(2515) ->  11 Load(24126) ->  18 Load(26958) ->  35 Load(29852) ->  0 Load(29852)
Distance of the route: 31505331277847289m
Load of the route: 29852

[1, 34, 2, 16, 3, 14, 18, 17, 13, 72, 7, 11, 6, 4, 9, 5, 8, 10, 1]
Route for vehicle 1:
 0 Load(0) ->  33 Load(10235) ->  1 Load(17298) ->  15 Load(17477) ->  2 Load(17528) ->  13 Load(17579) ->  17 Load(18107) ->  16 Load(18113) ->  12 Load(18170) ->  71 Load(20622) ->  6 Load(21669) ->  10 Load(22804) ->  5 Load(23153) ->  3 Load(23176) ->  8 Load(26177) ->  4 Load(29251) ->  7 Load(29949) ->  9 Load(29980) ->  0 Load(29980)
Distance of the route: 79469753026962279m
Load of the route: 29980

[1, 32, 33, 35, 61, 62, 59, 60, 64, 63, 65, 66, 67, 68, 70, 38, 39, 41, 69, 40, 58, 57, 42, 56, 55, 1]
Route for vehicle 2:
 0 Load(0) ->  31 Load(883) ->  32 Load(2572) ->  34 Load(2601) ->  60 Load(4548) ->  61